# Max Cut approach

After testing the NMcut, I wondered if Maxcut could be used directly to solve the splitting of the distance matrix.

---

Here, we'll make use of a distance matrix directly, so we need new functions to calculate it. However, as all the elements in the matrix are normalized between 0 and 100, we can directly invert the elements of all the matrices.

In [34]:
import os, sys, re, time
import numpy as np
sys.path.append('../')
from qa_functions import TreeNode, treecmp, Timer

In [ ]:
for i in range(8,40):
    
    os.makedirs(f'../distance_matrices/{i}', exist_ok=True)

file_names = os.listdir('../matrices')

for folder in file_names:
    files = os.listdir(f'../matrices/{folder}')
    for file in files:
        # Load the sequences
        distance_matrix = np.load(f'../matrices/{folder}/{file}')
        # Invert the matrix
        distance_matrix = 100 - distance_matrix
        # Save the inverted matrix
        np.save(f'../distance_matrices/{folder}/{file}', distance_matrix)

        del distance_matrix

---

We now have the distance matrix created from the alignments. Now we've got to define the new optimization problem using the max cut as a basis:

$$Max_{cut}=\sum_{i=1}^{n-1}\sum_{j=0}^{i-1}D_{ij}z_iz_j,\quad z_i\in \{-1,1\}.$$

We need to minimize this expression, therefore, our problems can be defined as:

$$\min Max_{cut}$$

In [5]:
print(distance_matrix)

[[100.          39.87122338  40.2281746   40.34482759  41.00861009
   39.31307141  41.67283494  63.93361407]
 [ 39.87122338 100.           2.73224044  39.61519487  40.82286277
   44.0732492   46.8248085   68.59340114]
 [ 40.2281746    2.73224044 100.          40.76086957  41.7221811
   45.13011152  47.73570898  68.09937888]
 [ 40.34482759  39.61519487  40.76086957 100.           6.00343053
   43.78538026  48.09535512  68.51714779]
 [ 41.00861009  40.82286277  41.7221811    6.00343053 100.
   46.01769912  50.22091311  70.2316412 ]
 [ 39.31307141  44.0732492   45.13011152  43.78538026  46.01769912
  100.          12.17948718  67.97029703]
 [ 41.67283494  46.8248085   47.73570898  48.09535512  50.22091311
   12.17948718 100.          70.09391992]
 [ 63.93361407  68.59340114  68.09937888  68.51714779  70.2316412
   67.97029703  70.09391992 100.        ]]


In [7]:
import dimod
from dimod import BinaryQuadraticModel, BINARY, SPIN
from dimod import ExactDQMSolver

distance_matrix = np.load('../distance_matrices/8/matrix_1822.npy')
J = {}
# Create the J dict for the BQM
for i in range(len(distance_matrix)):
    for j in range(i):
        if distance_matrix[i][j] != 0:
            J[(i, j)] = distance_matrix[i][j] 

model = BinaryQuadraticModel({},J,0.0,SPIN)
print(model)

BinaryQuadraticModel({1: 0.0, 0: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0}, {(0, 1): 39.87122337790986, (2, 1): 2.7322404371584668, (2, 0): 40.2281746031746, (3, 1): 39.615194869264926, (3, 0): 40.3448275862069, (3, 2): 40.76086956521739, (4, 1): 40.82286277408229, (4, 0): 41.00861008610086, (4, 2): 41.72218110041944, (4, 3): 6.00343053173242, (5, 1): 44.07324919574363, (5, 0): 39.31307141092167, (5, 2): 45.13011152416357, (5, 3): 43.78538026089097, (5, 4): 46.017699115044245, (6, 1): 46.824808500123545, (6, 0): 41.67283493708364, (6, 2): 47.73570898292502, (6, 3): 48.09535512410912, (6, 4): 50.220913107511045, (6, 5): 12.179487179487182, (7, 1): 68.59340114115604, (7, 0): 63.93361406985385, (7, 2): 68.09937888198758, (7, 3): 68.51714779175919, (7, 4): 70.23164120256284, (7, 5): 67.97029702970298, (7, 6): 70.09391992090954}, 0.0, 'SPIN')


In [11]:
var = []
    
for i in range(8):
    var.append(dimod.Binary(str(i)))
    
obj = BinaryQuadraticModel({},{},0.0,BINARY)

for i in range(8):
    for j in range(i):
        obj+=-distance_matrix[i,j]*(var[i]-var[j])**2
print(obj)

BinaryQuadraticModel({'1': -282.53298029543873, '0': -306.3723560712514, '2': -286.4086650950461, '3': -287.1222057291809, '4': -296.02733791745317, '5': -298.4692957159542, '6': -316.8230277521491, '7': -477.43940003793205}, {('0', '1'): 79.74244675581971, ('2', '1'): 5.4644808743169335, ('2', '0'): 80.4563492063492, ('3', '1'): 79.23038973852985, ('3', '0'): 80.6896551724138, ('3', '2'): 81.52173913043478, ('4', '1'): 81.64572554816458, ('4', '0'): 82.01722017220172, ('4', '2'): 83.44436220083888, ('4', '3'): 12.00686106346484, ('5', '1'): 88.14649839148726, ('5', '0'): 78.62614282184335, ('5', '2'): 90.26022304832713, ('5', '3'): 87.57076052178193, ('5', '4'): 92.03539823008849, ('6', '1'): 93.64961700024709, ('6', '0'): 83.34566987416729, ('6', '2'): 95.47141796585004, ('6', '3'): 96.19071024821824, ('6', '4'): 100.44182621502209, ('6', '5'): 24.358974358974365, ('7', '1'): 137.18680228231207, ('7', '0'): 127.8672281397077, ('7', '2'): 136.19875776397515, ('7', '3'): 137.0342955835

Ok, we have transformed the Ising $Max_{cut}$ formulation into a QUBO formulation, easier to work with. Also, using this QUBO formulation we can use the CQM solver.

In [23]:
test = dimod.ConstrainedQuadraticModel()
test.set_objective(obj)

solver = dimod.ExactCQMSolver()
sol = solver.sample_cqm(test)

# We want the best feasible solution. We can filter by its feasibility and take the first element
feas_sol = sol.filter(lambda s: s.is_feasible)
print(feas_sol.first.sample)

{'0': np.int64(0), '1': np.int64(1), '2': np.int64(1), '3': np.int64(1), '4': np.int64(1), '5': np.int64(0), '6': np.int64(0), '7': np.int64(0)}


We have one problem, this formulation seems to prefer balanced solutions, as expected, and this might not be the best, however, if we use the Ncut we arrive at the same problem as before. So let's create the architecture.

In [19]:
def max_cut (matrix:np.ndarray,tags=[])->BinaryQuadraticModel:
    r"""
    Creates a BinaryQuadraticModel from a numpy matrix using the Max-cut formulation. Both simmetrical matrices and matrices with 0 above the main diagonal work.
    
    Args:
        `matrix`: Matrix that defines the problem.
        `tags`: Optional list of tags for the variables. If not provided, the variables will be named as `Binary(0)`, `Binary(1)`, etc.
    
    Returns:
        The BinaryQuadraticModel from dimod that defines the problem.
    """
    
    rows = matrix.shape[0]
    var = []
    
    if not tags:
        for i in range(rows):
            var.append(dimod.Binary(i))
    else:
        var = [dimod.Binary(i) for i in tags]
    obj = BinaryQuadraticModel({},{},0.0,BINARY)
    
    for i in range(rows):
        for j in range(i):
            obj+=-matrix[i,j]*(var[i]-var[j])**2
            
    return obj

In [40]:
def bf_max_phylo_tree(matrix,tags=[],**kwargs):
    r"""
    Recursive function that uses D-Wave brute force solver to create the Phylogenetic tree using Ncut.
    
    Args:
        `matrix`: The matrix defining the graph.
        `tags`: Tags defining the names of the nodes, used for recursivity. **MUST BE AN INT LIST**
        
    Returns:
        The `TreeNode` containing the full tree. 
    """
    solver = dimod.ExactCQMSolver()
    
    if not tags:
        sub_mat = matrix
    else:
        sub_mat = matrix[np.ix_(tags, tags)]
        
    rows = sub_mat.shape[0]
    
    var = int(np.floor(rows/2.0))+1
    
    alpha = rows*100

    # Run max_cut
    
    if 'timer' in kwargs:
        start = time.time_ns()/1000000
        
    if not tags:
        problem = max_cut(sub_mat)
    else:
        problem = max_cut(sub_mat,tags=tags)
        
    model = dimod.ConstrainedQuadraticModel()
    model.set_objective(problem)
    sol = solver.sample_cqm(model)

    # We want the best feasible solution. We can filter by its feasibility and take the first element
    feas_sol = sol.filter(lambda s: s.is_feasible)
    
    if 'timer' in kwargs:
        end = time.time_ns()/1000000
        kwargs['timer'].update(end-start)
        
    n_graph_0 = [j for j in feas_sol.first.sample if feas_sol.first.sample[j]==0]
    n_graph_1 = [j for j in feas_sol.first.sample if feas_sol.first.sample[j]==1]
    # print(f'\tLa division es: {n_graph_0} | {n_graph_1}')
    
    node = TreeNode(tags)
    
    # Recursivity in the first graph
    if len(n_graph_0) > 2:
        if 'timer' in kwargs:
            node.children.append(bf_max_phylo_tree(matrix,n_graph_0,timer=kwargs['timer']))
        else:
            node.children.append(bf_max_phylo_tree(matrix,n_graph_0))
    else:
        leaf = TreeNode(n_graph_0)
        if len(n_graph_0) == 2:
            leaf.children.append(TreeNode([n_graph_0[0]]))
            leaf.children.append(TreeNode([n_graph_0[1]]))
        node.children.append(leaf)
        
    # Recursivity in the first graph
    if len(n_graph_1) > 2:
        if 'timer' in kwargs:
            node.children.append(bf_max_phylo_tree(matrix,n_graph_1,timer=kwargs['timer']))
        else:
            node.children.append(bf_max_phylo_tree(matrix,n_graph_1))
    else:
        leaf = TreeNode(n_graph_1)
        if len(n_graph_1) == 2:
            leaf.children.append(TreeNode([n_graph_1[0]]))
            leaf.children.append(TreeNode([n_graph_1[1]]))
        node.children.append(leaf)
    
    return node

In [33]:
tree = bf_max_phylo_tree(distance_matrix)
tree.print_tree()
tree.create_newick_file('testing.nwl')

	La division es: [0, 5, 6, 7] | [1, 2, 3, 4]
	La division es: [0, 7] | [5, 6]
	La division es: [3, 4] | [1, 2]
└── |
    ├── |
    │   ├── |
    │   │   ├── [0]
    │   │   └── [7]
    │   └── |
    │       ├── [5]
    │       └── [6]
    └── |
        ├── |
        │   ├── [3]
        │   └── [4]
        └── |
            ├── [1]
            └── [2]


In [35]:
with open(f'./testing.nwl','r') as fp:
    bf_tree = fp.read()

with open(f'../trees/8/tree_best_1822.newick','r') as fp:
    biotree = fp.read()
    

biotree = re.sub(r'taxon([0-9]+)',lambda match: str(int(match.group(1)) - 1),biotree)
biotree = re.sub(r':[0-9]\.[0-9]+(e-[0-9]+)*',r'',biotree)
# print(biotree)
dist = treecmp(bf_tree,biotree)
print(f'The Robinson-Foulds distance bewteen the Ncut with BF tree and the ground truth tree is \033[1m{dist}\033[22m')

The Robinson-Foulds distance bewteen the Ncut with BF tree and the ground truth tree is 100.0


Wow, this seems good, let's see what it does to all other trees.

In [41]:
file_names = os.listdir('../distance_matrices')

for folder in file_names:
    files = os.listdir(f'../distance_matrices/{folder}')
    if int(folder) > 16 and int(folder) < 25:
        for file in files:
            # Load the matrix
            distance_matrix = np.load(f'../distance_matrices/{folder}/{file}')

            print(f'Generating tree from: {file}')
            # Create the tree
            tree_bf = bf_max_phylo_tree(distance_matrix)

            file_ext = re.search(r'_([0-9]+)\.',file).group(1)
            new_file = f'max_bf_tree_{file_ext}.newick'
            tree_bf.create_newick_file(f'../trees/{folder}/{new_file}')

            del distance_matrix

Generating tree from: matrix_10618.npy
Generating tree from: matrix_41388.npy
Generating tree from: matrix_45255.npy
Generating tree from: matrix_48947.npy
Generating tree from: matrix_1.npy
Generating tree from: matrix_10240.npy
Generating tree from: matrix_29684.npy
Generating tree from: matrix_48363.npy
Generating tree from: matrix_11027.npy
Generating tree from: matrix_11191.npy


In [43]:
from collections import defaultdict
import os

file_names = os.listdir('../trees')
# with open(f'../metrics/RF-distance_bf_maxcut.csv','w+') as fp:
#     fp.write('size,id,RF\n')

# Remove the folder test
file_names = file_names[:-2]

for folder in file_names:
    files = os.listdir(f'../trees/{folder}')
    if int(folder) >7 and int(folder) < 16:
        trees = defaultdict(list)
        for file in files:
            name, ext = os.path.splitext(file)
            fields = name.split('_')
            # print(fields)
            if (fields[0] == 'max') or fields[0]=='tree':
                trees[fields[-1]].append(file)
        # print(trees)
        for key in trees.keys():
            print(f'Comparation between {trees[key][0]} and {trees[key][1]} is done')
            with open(f'../trees/{folder}/{trees[key][0]}','r') as fp:
                qa_tree = fp.read()

            with open(f'../trees/{folder}/{trees[key][1]}','r') as fp:
                biotree = fp.read()

            biotree = re.sub(r'taxon([0-9]+)',lambda match: str(int(match.group(1)) - 1),biotree)
            biotree = re.sub(r':[0-9]\.[0-9]+(e-[0-9]+)*',r'',biotree)
            # print(biotree)
            dist = treecmp(qa_tree,biotree)
            
            with open(f'../metrics/RF-distance_bf_maxcut.csv','a') as fp:
                fp.write(f'{folder},{key},{dist}\n')

            # print(f'The Robinson-Foulds distance bewteen the Ncut with QA tree and the Neighbor-Joining tree is \033[1m{dist}\033[22m')

Comparation between max_bf_tree_28864.newick and tree_best_28864.newick is done
Comparation between max_bf_tree_11523.newick and tree_best_11523.newick is done
Comparation between max_bf_tree_7791.newick and tree_best_7791.newick is done
Comparation between max_bf_tree_13951.newick and tree_best_13951.newick is done
Comparation between max_bf_tree_10953.newick and tree_best_10953.newick is done
Comparation between max_bf_tree_11018.newick and tree_best_11018.newick is done
Comparation between max_bf_tree_1822.newick and tree_best_1822.newick is done
Comparation between max_bf_tree_32709.newick and tree_best_32709.newick is done
